In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil
import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

import networkx as nx
import pandas as pd
import json

<IPython.core.display.Javascript object>

## Basic Setup
* Load a network from URL

In [2]:
cy = CyRestClient()

# Reset session
cy.session.delete()

network1 = cy.network.create_from('http://chianti.ucsd.edu/~kono/data/galFiltered.sif')

## Get Network View Object

In [3]:
view_id_list = network1.get_views()
print(view_id_list)

view1 = network1.get_view(view_id_list[0], format='view')

# This is a CyNetworkView object, not dictionary
print(view1)

view2 = network1.get_view(view_id_list[0], format='json')

# This contains Cytoscape.js style JSON as a Python dictionary
#print(view2)

[8819]


In [6]:
# Get views as Python dictionary
node_views_dict = view1.get_node_views_as_dict() 
edge_views_dict = view1.get_edge_views_as_dict() 

#print(json.dumps(node_views_dict, indent=4))
#print(json.dumps(edge_views_dict, indent=4))

{
    "8192": {
        "NODE_FILL_COLOR": "#89D0F5", 
        "NODE_NESTED_NETWORK_IMAGE_VISIBLE": true, 
        "NODE_LABEL_TRANSPARENCY": 255, 
        "NODE_LABEL_FONT_SIZE": 12, 
        "NODE_CUSTOMPAINT_1": "DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_1, name=Node Custom Paint 1)", 
        "NODE_BORDER_WIDTH": 0.0, 
        "NODE_CUSTOMPAINT_3": "DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_3, name=Node Custom Paint 3)", 
        "NODE_CUSTOMPAINT_2": "DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_2, name=Node Custom Paint 2)", 
        "NODE_CUSTOMPAINT_5": "DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_5, name=Node Custom Paint 5)", 
        "NODE_CUSTOMPAINT_4": "DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_4, name=Node Custom Paint 4)", 
        "NODE_CUSTOMPAINT_7": "DefaultVisualizableVisualProperty(id=NODE_CUSTOMPAINT_7, name=Node Custom Paint 7)", 
        "NODE_CUSTOMPAINT_6": "DefaultVisualizableVisualProperty(id=NODE_CUSTO

In [12]:
# Get views as NodeView/EdgeView objects
node_views = view1.get_node_views()
nv1 = node_views[0]
nv1_values = nv1.get_values()

#print(json.dumps(nv1_values, indent=4))

# Access single value
nv1_color = nv1.get_value('NODE_FILL_COLOR')
print(nv1_color)

#89D0F5


## Directly set Visual Property values 

In [28]:
node_ids = network1.get_nodes()

# Generate random colors for each node
import random

def generate_random_color(node_ids):
    new_values = {}
    for n in node_ids:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        new_color = '#' + hex(r)[2:] + hex(g)[2:] + hex(b)[2:]
        new_values[n] = new_color
    return new_values

for i in range(1,100):
    view1.update_node_views(visual_property='NODE_FILL_COLOR', values=generate_random_color(node_ids))

## Animation!

In [51]:
def set_size(node_ids, size):
    new_values = {}
    for n in node_ids:
        new_values[n] = size
    return new_values

import math
import time
base_size = 40

for i in range(1,200):
    view1.update_node_views(visual_property='NODE_HEIGHT', values=set_size(node_ids, base_size + math.sin(i/math.pi)*20 ))
    view1.update_node_views(visual_property='NODE_WIDTH', values=set_size(node_ids, base_size + math.sin(i/math.pi)*20 ))
    time.sleep(0.05)